In [ ]:
# get applicationids that have active video feedback (application settings page → SAM db)
# convert each applicationId to the correct guid
import cx_Oracle
import uuid

dsn_tns = cx_Oracle.makedsn('sit1oracle.prodlabs.local', '1521', service_name='GV')

def get_oracleid_from_bytes(byteString):
    return str(uuid.UUID(bytes=byteString).hex).upper()

# reference https://stackoverflow.com/questions/37684714/oracle-raw-in-string-format-to-standard-guid
def raw_to_guid(rawString):
    split_groups = [rawString[idx:idx + 2] for idx in range(0, len(rawString), 2)]

    return (split_groups[3]+split_groups[2]+split_groups[1]+split_groups[0]+'-'+split_groups[5]+split_groups[4]+'-'+split_groups[7]+
    split_groups[6]+'-'+split_groups[8]+split_groups[9]+'-'+split_groups[10]+split_groups[11]+split_groups[12]+split_groups[13]+split_groups[14]+split_groups[15])

try:
    with cx_Oracle.connect(user='sam', password='Password1', dsn=dsn_tns) as conn:
        with conn.cursor() as cursor:
            cursor.execute("select id from lk_settings where name='videoFeedback.enabled'")
            while True:
                row = cursor.fetchone()
                setting_id = get_oracleid_from_bytes(row[0])
                break
            application_ids = []
            cursor.execute("select applicationid from applicationsettings where settingid = :settingid and value='true'", [setting_id])
            while True:
                row = cursor.fetchone()
                if row is None:
                    break
                application_id = get_oracleid_from_bytes(row[0])
                application_id_guid = raw_to_guid(application_id)
                application_ids.append(application_id_guid)
            print(application_ids)

except cx_Oracle.Error as error:
    print(error)


In [10]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# limit the unsubscribed members to a specific timeframe (April 2021-today) --> VCC db
import pyodbc
import pandas as pd

cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=ag-listener1.prodlabs.local;"
            "Database=vcc;"
            "UID=vccuser;"
            "PWD=Password2Password2;")
cnxn = pyodbc.connect(cnxn_str)

data = pd.read_sql("select distinct IcmMemberId from vcc.dbo.auditevents where Succeeded=1 and CreateDate >= '2021-04-01' and CreateDate < '2022-11-01';", cnxn)
purgedMembersSince = set(data['IcmMemberId'].values)
print(purgedMembersSince)

In [ ]:
# get panelids corresponding to these applicationids (CPM db)
import pyodbc
import pandas as pd

cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=listener.icm-sit1-sqlserver-ee.cncjmkwlkqyp.us-west-2.rds.amazonaws.com;"
            "Database=cpm_sit;"
            "UID=cpm_user;"
            "PWD=Password1;")
cnxn = pyodbc.connect(cnxn_str)

panel_ids = []
tenant_ids = []
for appl_id in application_ids:
    data = pd.read_sql("select Id, TenantId from Panels where applicationId='{}'".format(appl_id), cnxn)
    panel_id = data['Id'].iloc[0]
    tenant_id = data['TenantId'].iloc[0]
    panel_ids.append(panel_id)
    tenant_ids.append(tenant_id)

print(tenant_ids)


In [ ]:
# generate list of memberIds who have unsubscribed (CPM db)
def get_purged_members_query(panel_id):
	return """\
	DECLARE @panelId AS uniqueidentifier = '{}';

	WITH PanelMembers AS (
		SELECT * FROM [cpm_sit].[dbo].Members WHERE panelId = @panelId
	)

	SELECT Id FROM PanelMembers
	WHERE
		MemberStatusSequenceId IN (
			SELECT mvv.SequenceId
			FROM [cpm_sit].[dbo].MemberVariableValues mvv
			JOIN [cpm_sit].[dbo].MemberVariables mv
				ON mvv.MemberVariableId = mv.Id
			WHERE
				mv.Name = 'memberstatus'
				AND mvv.[Key] = 'purged'
				AND mv.PanelId = @panelId
				AND mvv.PanelId = @panelId
		)
		AND MemberIdentity1SequenceId IN (
			SELECT mvv.SequenceId
			FROM [cpm_sit].[dbo].MemberVariableValues mvv
			JOIN [cpm_sit].[dbo].MemberVariables mv
				ON mvv.MemberVariableId = mv.Id
			WHERE
				mv.Name = 'email'
				AND mvv.TextValue LIKE '%@purged.disabled'
				AND mv.PanelId = @panelId
				AND mvv.PanelId = @panelId
		)""".format(panel_id)

panel_to_members = dict()

print('size of purged members: ', len(purgedMembersSince))

for panel_id in panel_ids:
	purgedMembersQuery = get_purged_members_query(panel_id)
	data = pd.read_sql(purgedMembersQuery, cnxn)
	panel_members = data['Id'].values.tolist()
	result = []
	for panel_member in panel_members:
		if panel_member in purgedMembersSince:
			result.append(panel_member)
	panel_to_members[panel_id] = result

print(panel_to_members)

In [ ]:
# convert tenant ids to oracle guid

# reference https://stackoverflow.com/questions/68886687/java-guid-convert-into-oracle-raw16-string 
def guid_to_raw(guidString):
    guidString = guidString.replace("-", "")
    split_groups = [guidString[idx:idx + 2] for idx in range(0, len(guidString), 2)]

    return (split_groups[3]+split_groups[2]+split_groups[1]+split_groups[0]+split_groups[5]+split_groups[4]+split_groups[7]+
    split_groups[6]+split_groups[8]+split_groups[9]+split_groups[10]+split_groups[11]+split_groups[12]+split_groups[13]+split_groups[14]+split_groups[15])

# generate list of respondentIds who have responded to and completed VPM (VCSD db)
dsn_tns = cx_Oracle.makedsn('sit1oracle.prodlabs.local', '1521', service_name='GMASTER')

tenant_to_respondents = dict()

try:
    with cx_Oracle.connect(user='sys', password='Password1', dsn=dsn_tns, mode=cx_Oracle.SYSDBA) as conn:
        with conn.cursor() as cursor:
            for tenant_id in tenant_ids:
                tenant_id = guid_to_raw(tenant_id)
                respondent_ids = []
                cursor.execute("""select distinct Resp.respondentid from firefly_rsp.response Resp inner join firefly_rsp.stepresponsedata StepResp on Resp.responseid = StepResp.responseid where StepResp.questiontype = 'VideoFeedbackPlugin' 
                and Resp.tenantid = :tenantid and Resp.completedat is not NULL and Resp.respondentid is not NULL and Resp.completedat >= TO_DATE('04/01/2021','MM/DD/YY') and Resp.completedat < TO_DATE('11/01/2022','MM/DD/YY') """, [tenant_id])
                while True:
                    row = cursor.fetchone()
                    if row is None:
                        break
                    respondent_id = row[0]
                    respondent_ids.append(respondent_id)
                tenant_to_respondents[tenant_id] = respondent_ids

except cx_Oracle.Error as error:
    print(error)

print(tenant_to_respondents)


In [ ]:
# map respondent ids to member ids
tenant_to_members = dict()

dsn_tns = cx_Oracle.makedsn('sit1oracle.prodlabs.local', '1521', service_name='GMASTER')

try:
    with cx_Oracle.connect(user='sys', password='Password1', dsn=dsn_tns, mode=cx_Oracle.SYSDBA) as conn:
        with conn.cursor() as cursor:
            for tenant_id in tenant_to_respondents:
                member_ids = []
                for respondent_id in tenant_to_respondents[tenant_id]:
                    cursor.execute("select publicid from firefly_app.respondent where respondentid= :respondentId", [respondent_id])
                    while True:
                        row = cursor.fetchone()
                        public_id = row[0].split(':')[2]
                        break
                    member_ids.append(public_id) 
                tenant_to_members[tenant_id] = member_ids

except cx_Oracle.Error as error:
    print(error)

print(tenant_to_members)


In [ ]:
# intersect the lists
from itertools import chain

def flatten_dict_values(dict):
    return [x.lower() for x in chain.from_iterable(dict.values())]


def intersect_lists(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

cpm_members = flatten_dict_values(panel_to_members)

vcsd_members = flatten_dict_values(tenant_to_members)

found_members = intersect_lists(cpm_members, vcsd_members)
print(found_members)

In [ ]:
# map member ids back to panel ids

cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=listener.icm-sit1-sqlserver-ee.cncjmkwlkqyp.us-west-2.rds.amazonaws.com;"
            "Database=cpm_sit;"
            "UID=cpm_user;"
            "PWD=Password1;")
cnxn = pyodbc.connect(cnxn_str)

found_panel_to_members = dict()
for member_id in found_members:
    data = pd.read_sql("select PanelId from Members where Id='{}'".format(member_id), cnxn)
    panel_id = data['PanelId'].iloc[0]
    panel_members = found_panel_to_members.get(panel_id, [])
    panel_members.append(member_id)
    found_panel_to_members[panel_id] = panel_members

print(found_panel_to_members)
